# Dependences

In [9]:
include("../../src/struct_data.jl")

save_data

# Model

In [10]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000003 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [11]:
# Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.06)
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.02)
agg_size = 15

15

# Aggregate

In [12]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.03)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 

In [13]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [14]:
include("../../src/neighbor.jl")

dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

display(agg.Simulation.Neighbor.idx_red)

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  0   0   0   0  59  44  45   0   0  …  2462  2463     0     0     0     0
  0   0   0   0   0  60  61   0   0     2445  2446     0     0     0     0
  0   0   0   0   0   0   0   0   0        0    

# Sum Forces

In [15]:
include("../../src/forces/forces.jl")

threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Force.N_i,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        pi/4,
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)

  0.739565 seconds (373.76 k CPU allocations: 21.756 MiB)


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.000307647   0.0340083     0.069724
  0.0823097     0.0548668     0.00644826
  0.00323512    0.0274034    -0.0416916
  0.0206936     0.00848406   -0.0519423
  0.0630597    -0.0565954    -0.0193601
 -0.0295293    -0.0657677     0.00142192
  0.0196277     0.00136373   -0.015307
 -0.00196592    0.029465     -0.0373971
 -0.0127427     0.0269269     0.00308069
  0.0308776    -0.00103023   -0.0229675
  0.0583973    -0.0492944    -0.0523347
  0.0885777    -0.0749552    -0.00414274
 -0.0223067    -0.0847059     0.053305
  ⋮                          
 -0.0205575     0.0460396    -0.00606437
  0.0547175     0.00669029   -0.0246493
  0.00369423    0.0748939     0.00162859
 -3.38513f-5    9.98547f-5   -0.000364828
 -0.0221222     0.0568284    -0.030251
 -0.0509051    -0.0468388    -0.0127709
 -0.0422138     0.0310854     0.027842
 -4.6549f-12   -0.000164602  -0.000364654
 -0.0380546     0.010274     -0.042187
  0.00366509    0.0228036     0.015

# MiniTest

In [16]:
include("../../src/run_event.jl")

run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing... 100%|████████████████████████████████████| Time: 0:01:31


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0344191    0.00563465  -0.0265658
 -0.0180111   -0.0269911   -0.0249067
 -0.064242    -0.0239065    0.0260628
  0.0256945    0.0550145    0.0582419
  0.0485815   -0.0310903    0.0292934
 -0.0709806   -0.0465179   -0.00958036
  0.00856424  -0.0278905   -0.0178473
 -0.0342739    0.0203587    0.0471882
  0.0691532    0.0477399    0.0379233
 -0.0262772    0.0187961    0.0807334
  0.0109733   -0.00959783  -0.084803
  0.00588632   0.0225194   -0.0203873
 -0.030951     0.00637068  -0.0093293
  ⋮                        
  0.0582522    0.0162912   -0.0407579
 -0.012688     0.023865    -0.051927
  0.00203545  -0.0332672    0.0105603
  0.0202084    0.0593706    0.00693523
  0.0499056    0.0311456   -0.0428085
  0.00646757  -0.0762123   -0.0341487
 -0.0597207   -0.0238247   -0.0457419
  0.00145121   0.0134189   -0.00869056
  0.0237068   -0.00344613   0.00563613
 -0.00183232   0.0551754   -0.0899966
  5.38046f-5  -0.0548942   -0.0995968
 -0.003

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  -6.95308    -9.50279   -0.804944
  -3.10234   -13.3703    -0.13384
   7.28385    -7.40565   -0.963261
  -8.73155    -6.7773     2.39326
  -4.70747   -14.9227     0.359993
  -1.72672    -6.33546    0.871262
   3.13067    -6.34448    3.76896
  -2.18775    -3.71816   -0.419053
  -9.29764   -10.8151     2.03675
  -9.04524    -5.24075   -0.786008
 -11.7752     -3.91499    0.675004
  -4.62263    -7.35286   -0.587805
   9.43353    -5.35368   -1.93628
   ⋮                     
  -2.22306    -1.46769    9.49428
   2.15471    -1.39051   11.1826
   0.403655    1.12351   12.782
   3.89534    -1.66055   11.4606
  -4.52301     1.12783    9.09962
  -5.29367     1.72069   10.5676
  -0.907955   -3.06316    9.48091
  -1.07738     0.114342   9.72478
   2.65795     1.39356   12.1105
  -3.55441     6.10419   10.5605
  -0.650424   -2.71878   11.6087
   2.20586     2.62992   10.5676